#### The code below is used to audit, clean and update the street names

In [ ]:
OSMFILE = "des_moines.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle"]


In [ ]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r", encoding="utf8")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))


In [ ]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "Rd.,": "Road",
            "Ave": "Avenue",
            "ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Cir": "Circle",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Hwy":"Highway",
            "Ct": "Court",
            "Pkwy": "Parkway",
            "Ln": "Lane",
            "Sq": "Square",
            "Ste": "Suite",
            "Real":"Real",
            "Saratoga":"Saratoga",
            "Alameda": "Alameda",
            "Way": "Way",
            "Expressway": "Expressway"
            }

def update_name(name, mapping):
    
    m = street_type_re.search(name)
    new_name = name
    if m:
        street_type = m.group()
        if street_type in mapping:
            new_name = street_type_re.sub(mapping[street_type], name)  
    return new_name

for st_type, ways in st_types.items(): # changed this method from .iteritems() from 2.7 to 3.6's .items()  
        for name in ways:
            better_name = update_name(name, mapping)
            print (name, "=>", better_name)